In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
os.getcwd()

# Note email cpm updated based on the Q1 client number
# Flipp hosted not included in wide version

'/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2'

In [2]:
new_quarter="2020Q2"
new_quarter_start_date="2020-05-03"
new_quarter_end_date="2020-08-01"
os.getcwd()

'/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2'

In [3]:
TMR_up_to_2020Q1=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/output/BL_MMM_long_cumu_up_to_2020Q1_JL_2020-12-16.csv",dtype=str)
TMR_up_to_2020Q1['impression']=TMR_up_to_2020Q1['impression'].fillna(0).astype(float).astype(int)
TMR_up_to_2020Q1['click']=TMR_up_to_2020Q1['click'].fillna(0).astype(float).astype(int)
TMR_up_to_2020Q1['cost']=TMR_up_to_2020Q1['cost'].fillna(0).astype(float)


In [4]:
TMR_up_to_2020Q1_no_DMA=TMR_up_to_2020Q1.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2020Q1_no_DMA['week date'].min())
print(TMR_up_to_2020Q1_no_DMA['week date'].max())
# TMR_up_to_2020Q1_no_DMA

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


2016-09-25
2020-04-26


In [5]:
TMR_up_to_2020Q1_no_DMA.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2016-09-25,Email,xx,xx,11581550,113834,5278.691191
1,2016-10-02,Digital,Flipp,Flipp App,74691,37222,10794.380000


In [6]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

# TV

In [7]:
# Email: Big Lots Q2 TMR from Jessica, Fhursday, Sept 9, 2020 at 3:28 PM
'''
Media   (net)                                        
National&Sinclair $1,106,458
DRTV $384,878
'''

# Only 1 week for local sinclair, ignore it
NationalSinclair_tv_cost=1106458
DRTV_tv_cost=384878
TV_TMR_data=pd.DataFrame()

In [8]:
TV_print_excel=pd.ExcelFile("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/Big Lots Q2 TMR 9.9.20.xlsx")
print(TV_print_excel.sheet_names)
                      
df_TV_national=TV_print_excel.parse("National",dtype=str,skiprows=2)
# df_TV_sinclair=TV_print_excel.parse("Sinclair",dtype=str,skiprows=2)
df_TV_DRTV=TV_print_excel.parse("DRTV",dtype=str,skiprows=2)

print(df_TV_national.shape,df_TV_DRTV.shape)


['National', 'DRTV', 'Circ Spend']
(2720, 18) (2958, 18)


In [9]:
df_TV_national.head(2)

,Q1,Post,Strata: NHI ACM Live +3,Baltimore,WeTV,Spot Cable,W25-54,2018-02-05 00:00:00,2/10/2018,Saturday,5:27 PM,30,LIVE PD,21.25,0.77,3935,"6740 COMCAST, Baltimore Interconnect-Cable",Unnamed: 17
0,Q2 2020,Post,Strata: NHI ACM Live +3,National,BRVO,Cable,W25-54,2020-05-11 00:00:00,05/17/2020,Su,3:25AM,:15,CASH CAB,NaN,0.04189048239895697,25704,NaN,NaN
1,Q2 2020,Post,Strata: NHI ACM Live +3,National,CNN,Cable,W25-54,2020-05-11 00:00:00,05/17/2020,Su,6:16AM,:15,NEW DAY SUNDAY,NaN,0.02770208604954368,16998,NaN,NaN


In [10]:
df_TV_national=df_TV_national[["Q1",'Baltimore',"WeTV",' 2/10/2018',3935]]
df_TV_national.columns=["Q1 2020",'National',"Network","Air Date",' Imp(000) - Women 25-54']
# df_TV_national['Air Date']=df_TV_national['Air Date'].apply(lambda x: x.strip())
df_TV_national=df_TV_national[pd.notnull(df_TV_national['Air Date'])]

df_TV_national=df_TV_national.rename(columns={"Network":"placement"})
df_TV_national['date']=df_TV_national['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_national['impression']=df_TV_national[' Imp(000) - Women 25-54'].astype(float)

df_TV_national['media']="TV"
df_TV_national['submedia']="National"
df_TV_national['click']=0
df_TV_national['week date']=df_TV_national['date'].apply(get_week_end_date_BL)
df_TV_national['week date']=df_TV_national['week date'].apply(lambda x: x-datetime.timedelta(days=6))

sum_national_cable_impr=df_TV_national['impression'].sum()
df_TV_national['cost']=df_TV_national['impression']/sum_national_cable_impr*NationalSinclair_tv_cost
print("df_TV_national['cost'].sum()",df_TV_national['cost'].sum())
df_TV_national=df_TV_national[TMR_up_to_2020Q1_no_DMA.columns.tolist()]


df_TV_national['cost'].sum() 1106458.0


In [11]:
df_TV_DRTV.head(2)

,Q1,Post,Unnamed: 2,Baltimore,WeTV,Spot Cable,W25-54,2018-02-05 00:00:00,2/10/2018,Saturday,5:27 PM,30,LIVE PD,21.25,0.77,3935,"6740 COMCAST, Baltimore Interconnect-Cable",Unnamed: 17
0,Q2 2020,Post,Strata: NHI ACM Live +3,National,Fox News Channel,Cable,W25-54,2020-06-15 00:00:00,2020-06-21 00:00:00,Sunday,06:59A,:15,FOX AND FRIENDS,NaN,0.06,36714,NaN,NaN
1,Q2 2020,Post,Strata: NHI ACM Live +3,National,Fox News Channel,Cable,W25-54,2020-06-15 00:00:00,2020-06-21 00:00:00,Sunday,07:43A,:15,FOX AND FRIENDS,NaN,0.09,55071,NaN,NaN


In [12]:
df_TV_DRTV=df_TV_DRTV[["Q1",'Baltimore',"WeTV",' 2/10/2018',3935]]
df_TV_DRTV.columns=["Q1 2020",'National',"Network","Air Date",' Imp(000) - Women 25-54']
# df_TV_DRTV['Air Date']=df_TV_DRTV['Air Date'].apply(lambda x: x.strip())
df_TV_DRTV=df_TV_DRTV[pd.notnull(df_TV_DRTV['Air Date'])]

df_TV_DRTV=df_TV_DRTV.rename(columns={"Network":"placement"})
df_TV_DRTV['date']=df_TV_DRTV['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_DRTV['impression']=df_TV_DRTV[' Imp(000) - Women 25-54'].astype(float)

df_TV_DRTV['media']="TV"
df_TV_DRTV['submedia']="National"
df_TV_DRTV['click']=0
df_TV_DRTV['week date']=df_TV_DRTV['date'].apply(get_week_end_date_BL)
df_TV_DRTV['week date']=df_TV_DRTV['week date'].apply(lambda x: x-datetime.timedelta(days=6))

sum_DRTV_impr=df_TV_DRTV['impression'].sum()
df_TV_DRTV['cost']=df_TV_DRTV['impression']/sum_DRTV_impr*DRTV_tv_cost
print("df_TV_DRTV['cost'].sum()",df_TV_DRTV['cost'].sum())
df_TV_DRTV=df_TV_DRTV[TMR_up_to_2020Q1_no_DMA.columns.tolist()]


df_TV_DRTV['cost'].sum() 384878.0


In [13]:
TV_TMR_data=df_TV_national.append(df_TV_DRTV)
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)
print("TV_TMR_data['cost'].sum()",TV_TMR_data['cost'].sum())
print("TV_TMR_data['impression'].sum()",TV_TMR_data['impression'].sum())


TV_TMR_data['cost'].sum() 1491336.0
TV_TMR_data['impression'].sum() 30574214.5


In [14]:
TV_TMR_data.shape

(3994, 7)

In [15]:
TV_TMR_data=TV_TMR_data.groupby(['week date','media','submedia','placement'])['impression','click','cost'].sum().reset_index()
TV_TMR_data.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


(333, 7)

In [16]:
TV_TMR_data=TV_TMR_data[TV_TMR_data['week date']>=new_quarter_start_date]
TV_TMR_data=TV_TMR_data[TV_TMR_data['week date']<=new_quarter_end_date]
print(TV_TMR_data.shape)
print(TV_TMR_data['week date'].nunique())
print(TV_TMR_data['week date'].min())
print(TV_TMR_data['week date'].max())


df_TV_summary=TV_TMR_data.groupby(['submedia'])['impression','cost'].sum().reset_index()
df_TV_summary['impression']=df_TV_summary['impression'].astype(int)
df_TV_summary

(333, 7)
4
2020-05-17
2020-06-28


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


,submedia,impression,cost
0,National,30574214,1491336.0


In [17]:
df_qc_tv_lastyear=TMR_up_to_2020Q1_no_DMA[TMR_up_to_2020Q1_no_DMA['week date']>="2019-05-05"]
df_qc_tv_lastyear=df_qc_tv_lastyear[df_qc_tv_lastyear['week date']<="2019-08-03"]
df_qc_tv_lastyear=df_qc_tv_lastyear[df_qc_tv_lastyear['media']=="TV"]
print(df_qc_tv_lastyear['cost'].sum(),df_qc_tv_lastyear['impression'].sum())

# In line

1697094.7 111259722


In [18]:
df_TV_summary['cost'].sum(),df_TV_summary['impression'].sum()

(1491336.0000000002, 30574214)

In [19]:
1491336.0000000002/1697094.7

0.878758268469049

In [20]:
30574214/111259722

0.2748003810399598

In [21]:
print(datetime.datetime.now())

2020-12-16 21:52:55.642754


# Print Circulation

In [22]:
week_1=datetime.date(2020,2,2)
df_week_2020Q2=pd.DataFrame({"week date":week_1},index=range(13,26))
df_week_2020Q2['Week']=range(13,26)
df_week_2020Q2['week date']=df_week_2020Q2['Week'].apply(lambda x: week_1+datetime.timedelta(days=x*7))
df_week_2020Q2['Week']=df_week_2020Q2['Week']+1

In [23]:
df_print_circ=TV_print_excel.parse("Circ Spend",dtype=str,skiprows=1)
df_print_circ['Week']=df_print_circ['Week'].apply(lambda x: int(x.split("(")[0].split(" ")[1]))
df_print_circ=pd.merge(df_print_circ,df_week_2020Q2,on="Week")
df_print_circ=df_print_circ.rename(columns={"Quantities":"impression","Spend":"cost"})
df_print_circ['click']=0
df_print_circ['cost']=df_print_circ['cost'].astype(float)
df_print_circ['impression']=df_print_circ['impression'].astype(int)

df_print_circ['media']="Circulation"
df_print_circ['submedia']="xx"
df_print_circ['placement']="xx"

df_print_circ=df_print_circ[TMR_up_to_2020Q1_no_DMA.columns.tolist()]
print(df_print_circ.shape)
print(df_print_circ['week date'].nunique(),df_print_circ['week date'].min(),df_print_circ['week date'].max())
print(pd.to_datetime(df_print_circ['week date']).dt.day_name().unique())

# Update for this quarter some circulation was printed but not delivered
df_print_circ['cost']=np.where(df_print_circ['impression']==0,0,df_print_circ['cost'])

(5, 7)
5 2020-05-17 2020-07-26
['Sunday']


# Others from Datorama

In [24]:
list_datorama_files=glob.glob("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/datorama_files/*.csv")
list_datorama_files.sort()
list_datorama_files

['/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/datorama_files/Digital Weekly Media Data  (Search).csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/datorama_files/Digital Weekly Media Data without Placement (excluded FB).csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/datorama_files/E-circular_Flipp & Print Circular & Email by Week.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/input_2020Q2/datorama_files/Social Daily Media Data.csv']

In [25]:
df_1_search=pd.read_csv(list_datorama_files[0])
df_1_search.columns=[x.lower() for x in df_1_search.columns.tolist()]
df_1_search=df_1_search.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_1_search=df_1_search[['week','media','submedia','placement','impression','click','cost']]
df_1_search['submedia']=df_1_search['submedia'].replace({"Paid SEM Campaign":"SEM"}).replace({"Shopping Campaign":"PLA"})
df_1_search=df_1_search.sort_values(['media','submedia','placement','week'])
df_1_search.head(2)

,week,media,submedia,placement,impression,click,cost
6,2020-05-17 00:00,Digital,PLA,Bing,1731.0,40.0,21.11
16,2020-05-31 00:00,Digital,PLA,Bing,618901.0,12864.0,2586.76


In [26]:
# some weeks for The Reddit programmatic are broken down, cost and (impr + click) are seperated to different rows
# Doesn't matter since gonna rolling up

df_2_digital=pd.read_csv(list_datorama_files[1])
df_2_digital.columns=[x.lower() for x in df_2_digital.columns.tolist()]
df_2_digital=df_2_digital.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_2_digital=df_2_digital[['week','media','submedia','impression','click','cost','partner','tactic']]

df_2_digital_YT=df_2_digital[df_2_digital['partner']=="YouTube"]
df_2_digital_YT['placement']=df_2_digital_YT['submedia']
df_2_digital_YT['submedia']="YouTube"
df_2_digital_YT=df_2_digital_YT[df_1_search.columns.tolist()]

df_2_digital_Pandora=df_2_digital[df_2_digital['partner']=="Pandora"]
df_2_digital_Pandora['placement']=df_2_digital_Pandora['submedia']
df_2_digital_Pandora['submedia']="Pandora"
df_2_digital_Pandora=df_2_digital_Pandora[df_1_search.columns.tolist()]

df_2_digital_Programmatic=df_2_digital[~df_2_digital['partner'].isin(["YouTube",'Pandora'])]
df_2_digital_Programmatic['placement']=df_2_digital_Programmatic['submedia']
df_2_digital_Programmatic['submedia']="Programmatic"
df_2_digital_Programmatic=df_2_digital_Programmatic[df_1_search.columns.tolist()]
df_2_digital_Programmatic['cost']=df_2_digital_Programmatic['cost'].fillna(0)
df_2_digital_Programmatic['impression']=df_2_digital_Programmatic['impression'].fillna(0)
df_2_digital_Programmatic['click']=df_2_digital_Programmatic['click'].fillna(0)
df_2_digital_Programmatic['media']=df_2_digital_Programmatic['media'].fillna("Digital")

df_2_digital=df_2_digital_YT.append(df_2_digital_Pandora).append(df_2_digital_Programmatic)


df_2_digital.head(2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,week,media,submedia,placement,impression,click,cost
0,2020-07-26 00:00,Digital,YouTube,Programmatic Video,25442562.0,15519.0,0.000000
8,2020-07-26 00:00,Digital,YouTube,Multiplatform Video,0.0,0.0,143386.320385


In [27]:
df_3_email_ecirc=pd.read_csv(list_datorama_files[2])
df_3_email_ecirc.columns=[x.lower() for x in df_3_email_ecirc.columns.tolist()]
df_3_email_ecirc=df_3_email_ecirc.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_3_email_ecirc=df_3_email_ecirc[['week','media','placement','impression','click','cost']]
df_3_email_ecirc.insert(2,"submedia",np.nan)
df_3_email_ecirc['submedia']=np.where(df_3_email_ecirc['media']=="Digital","Flipp","xx")
df_3_email_ecirc['placement']=np.where(df_3_email_ecirc['media']=="Digital",df_3_email_ecirc['placement'],"xx")

df_3_email_ecirc.head(3)

,week,media,submedia,placement,impression,click,cost
0,2020-07-26 00:00,Email,xx,xx,187054273.0,1351913.0,85256.441758
1,2020-07-26 00:00,Digital,Flipp,Hosted,157520.0,134286.0,0.000000
2,2020-07-19 00:00,Email,xx,xx,157237129.0,1512856.0,71666.249136


In [28]:
df_4_social=pd.read_csv(list_datorama_files[3])
df_4_social.columns=[x.lower() for x in df_4_social.columns.tolist()]
df_4_social=df_4_social.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})

df_4_social=df_4_social[['week','media','submedia','partner','impression','click','cost']]
df_4_social=df_4_social.rename(columns={"partner":"placement"})
df_4_social.head(3)

,week,media,submedia,placement,impression,click,cost
0,2020-05-03 00:00,Digital,Social,Twitter,5564047.0,17419.0,30400.00
1,2020-05-03 00:00,Digital,Social,Pinterest,3430553.0,7721.0,9651.54
2,2020-05-03 00:00,Digital,Social,Facebook,2005882.0,48539.0,10676.81


In [29]:
df_all_digital_from_datorama=pd.concat([df_1_search,df_2_digital,df_3_email_ecirc,df_4_social])
df_all_digital_from_datorama['week']=df_all_digital_from_datorama['week'].apply(lambda x: x.split(" ")[0])
df_all_digital_from_datorama=df_all_digital_from_datorama.rename(columns={"week":"week date"})

# QC week date as 
df_all_digital_from_datorama['week_as_date'] = pd.to_datetime(df_all_digital_from_datorama['week date'])
df_all_digital_from_datorama['day_of_week'] = df_all_digital_from_datorama['week_as_date'].dt.day_name()
print(df_all_digital_from_datorama['day_of_week'].unique())
del df_all_digital_from_datorama['week_as_date']
del df_all_digital_from_datorama['day_of_week']

df_all_digital_from_datorama.head(2)

['Sunday']


,week date,media,submedia,placement,impression,click,cost
6,2020-05-17,Digital,PLA,Bing,1731.0,40.0,21.11
16,2020-05-31,Digital,PLA,Bing,618901.0,12864.0,2586.76


In [30]:
print(df_all_digital_from_datorama['week date'].nunique())
print(df_all_digital_from_datorama['week date'].min())
print(df_all_digital_from_datorama['week date'].max())

13
2020-05-03
2020-07-26


# Cumulative

In [31]:
Q2_2020_only_TMR=df_all_digital_from_datorama.append(df_print_circ).append(TV_TMR_data)
Q2_2020_only_TMR['week date']=Q2_2020_only_TMR['week date'].astype(str)
print(Q2_2020_only_TMR['week date'].nunique())
print(Q2_2020_only_TMR['week date'].min())
print(Q2_2020_only_TMR['week date'].max())


13
2020-05-03
2020-07-26


In [32]:
# Update email cost based on the confirmation from the client

# Email:  BL QC of non-Media Storm media for 2019 Q2 from Bryan Kipp, 2020-12-15 5:29 PM
Q2_2020_only_TMR_email=Q2_2020_only_TMR[Q2_2020_only_TMR['media']=="Email"]

cost_q1_total=445849
cost_q1_impression=Q2_2020_only_TMR_email['impression'].sum()
print(Q2_2020_only_TMR_email.shape)
Q2_2020_only_TMR_email['cost']=Q2_2020_only_TMR_email['impression']*cost_q1_total/cost_q1_impression

print(Q2_2020_only_TMR_email['impression'].sum(),Q2_2020_only_TMR_email['click'].sum(),Q2_2020_only_TMR_email['cost'].sum())


(13, 7)
2179046742.0 21905243.0 445849.0


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [33]:
Q2_2020_only_TMR_others=Q2_2020_only_TMR[Q2_2020_only_TMR['media']!="Email"]
Q2_2020_only_TMR=Q2_2020_only_TMR_others.append(Q2_2020_only_TMR_email)
#
Cum_Q2_2020_TMR=TMR_up_to_2020Q1.append(Q2_2020_only_TMR)

In [34]:
try:
    os.stat("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/output/")
except:
    os.mkdir("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/output/")

Q2_2020_only_TMR.to_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/output/BL_MMM_2020Q2_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q2_2020_TMR.to_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2/output/BL_MMM_long_cumu_up_to_2020Q2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


# Wide

In [35]:
# seperate the hosted flipp
Cum_Q2_2020_TMR_FlippHost=Cum_Q2_2020_TMR[(Cum_Q2_2020_TMR['submedia']=="Flipp") & (Cum_Q2_2020_TMR['placement']=="Hosted")]
Cum_Q2_2020_TMR_FlippHost['submedia']="Flipp(hosted)"

Cum_Q2_2020_TMR_FlippNonHost=Cum_Q2_2020_TMR[(Cum_Q2_2020_TMR['submedia']=="Flipp") & (Cum_Q2_2020_TMR['placement']!="Hosted")]
Cum_Q2_2020_TMR_FlippNonHost['submedia']="Flipp(nonhosted)"

Cum_Q2_2020_TMR=Cum_Q2_2020_TMR[(Cum_Q2_2020_TMR['submedia']!="Flipp")]
Cum_Q2_2020_TMR['submedia']=np.where(Cum_Q2_2020_TMR['media']=="Circulation","xx",Cum_Q2_2020_TMR['submedia'])

Cum_Q2_2020_TMR=Cum_Q2_2020_TMR.append(Cum_Q2_2020_TMR_FlippHost).append(Cum_Q2_2020_TMR_FlippNonHost)
########################
Cum_Q2_2020_TMR['week date']=Cum_Q2_2020_TMR['week date'].astype(str)

data_media_national=Cum_Q2_2020_TMR.groupby(['week date','media'])['impression','click','cost'].sum().reset_index()
data_submedia_national=Cum_Q2_2020_TMR.groupby(['week date','media','submedia'])['impression','click','cost'].sum().reset_index()
data_submedia_national['media_submedia']=data_submedia_national['media']+"-"+data_submedia_national['submedia']
data_submedia_national=data_submedia_national[['week date','media_submedia','impression','click','cost']]


data_media_national_wide=data_media_national[['week date']].drop_duplicates()
for col in data_media_national.columns.tolist()[-3:]:
    df=data_media_national[['week date','media']+[col]].pivot(index='week date',columns='media',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_media_national_wide=pd.merge(data_media_national_wide,df,on='week date',how="outer")
data_media_national_wide=data_media_national_wide.fillna(0)
data_media_national_wide=data_media_national_wide.drop_duplicates()
#############

data_submedia_national_wide=data_submedia_national[['week date']].drop_duplicates()
for col in data_submedia_national.columns.tolist()[-3:]:
    df=data_submedia_national[['week date','media_submedia']+[col]].pivot(index='week date',columns='media_submedia',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_submedia_national_wide=pd.merge(data_submedia_national_wide,df,on='week date',how="outer")
data_submedia_national_wide=data_submedia_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.drop_duplicates()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  from ip

In [36]:
including_weeks=[str(datetime.datetime.strptime(new_quarter_end_date,"%Y-%m-%d").date()-datetime.timedelta(days=7*x+6)) for x in range(104)]


data_media_national_wide=data_media_national_wide.sort_values("week date")
data_media_national_wide=data_media_national_wide[data_media_national_wide['week date'].isin(including_weeks)]

data_submedia_national_wide=data_submedia_national_wide.sort_values("week date")
data_submedia_national_wide=data_submedia_national_wide[data_submedia_national_wide['week date'].isin(including_weeks)]



In [37]:
data_submedia_national_wide

,week date,Cinema-xx_impression,Circulation-xx_impression,Digital-Flipp(hosted)_impression,Digital-Flipp(nonhosted)_impression,Digital-PLA_impression,Digital-Pandora_impression,Digital-Programmatic_impression,Digital-SEM_impression,Digital-Social_impression,...,Digital-PLA_cost,Digital-Pandora_cost,Digital-Programmatic_cost,Digital-SEM_cost,Digital-Social_cost,Digital-YouTube_cost,Email-xx_cost,Radio-Radio_cost,TV-Local_cost,TV-National_cost
97,2018-08-05,0.0,0.0,133902.0,542.0,3912625.0,390810.0,2061801.0,1052808.0,4038136.0,...,11514.23,8538.11,21119.170000,50169.71,19965.503081,18436.600000,40738.739038,0.0,0.0,0.000000
98,2018-08-12,0.0,0.0,153357.0,0.0,4505762.0,318803.0,4228711.0,1202253.0,13221622.0,...,15254.57,6951.19,33990.720000,54516.66,110546.017558,18121.960000,47208.774389,0.0,0.0,44723.796000
99,2018-08-19,0.0,13481187.0,303785.0,0.0,3830969.0,510792.0,5861033.0,985952.0,11660601.0,...,10066.16,10784.37,37662.690000,42902.13,100863.723102,20711.360000,52619.171353,0.0,0.0,139026.948500
100,2018-08-26,0.0,0.0,283738.0,375173.0,4423629.0,609882.0,5275191.0,1514057.0,26262856.0,...,12960.11,12429.37,50490.960000,90583.24,168745.379220,51034.500000,42962.910060,0.0,0.0,640585.074000
101,2018-09-02,0.0,15875846.0,182567.0,267846.0,4517185.0,751662.0,7987096.0,1757993.0,23399554.0,...,11776.92,15004.95,63668.740000,90905.15,137753.829514,63389.110000,42065.481738,0.0,0.0,576567.574500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,2020-06-28,0.0,0.0,182370.0,0.0,18282030.0,0.0,10200208.0,2001687.0,29811115.0,...,35965.59,0.00,109305.768924,55279.26,123733.220000,63347.590619,33982.816681,0.0,0.0,185556.918643
197,2020-07-05,0.0,0.0,215356.0,0.0,17934945.0,0.0,16410864.0,1928941.0,33754799.0,...,41728.75,0.00,113669.580322,55028.07,170533.600000,84772.788760,33179.015238,0.0,0.0,0.000000
198,2020-07-12,0.0,11960954.0,164553.0,0.0,17365127.0,0.0,26412731.0,1924813.0,41353748.0,...,43513.98,0.00,131026.736747,55831.70,308148.520000,157572.087318,30836.978530,0.0,0.0,0.000000
199,2020-07-19,0.0,0.0,191881.0,0.0,16321757.0,0.0,51248261.0,2015274.0,63261297.0,...,39486.64,0.00,275664.842152,59921.51,392391.020000,235276.327651,32171.873772,0.0,0.0,0.000000


# Sales

In [38]:
sales_data=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Automation/Weekly_Operation_Reports/outputs/combined_sales_long_2020-11-28.csv",dtype=str)
sales_data=sales_data[sales_data['week_end_date']<=new_quarter_end_date]
sales_data['week_end_date']=sales_data['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_data['sales']=sales_data['sales'].astype(float)
sales_data['transactions']=sales_data['transactions'].astype(float)
sales_data['week date']=sales_data['week_end_date'].apply(lambda x: x-datetime.timedelta(days=6))
sales_data_naitonal=sales_data.groupby(['week date'])['sales','transactions'].sum().reset_index()
sales_data_naitonal=sales_data_naitonal.set_index(['week date'])
sales_data_naitonal.at[datetime.date(2017,4,30),'transactions']=(sales_data_naitonal.at[datetime.date(2017,4,23),'transactions']+sales_data_naitonal.at[datetime.date(2017,5,7),'transactions'])/2
sales_data_naitonal=sales_data_naitonal.reset_index()
sales_data_store_counts=sales_data[sales_data['sales']>0]
sales_data_store_counts=sales_data_store_counts.groupby(['week date'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_counts"})

sales_data_naitonal=pd.merge(sales_data_naitonal,sales_data_store_counts,on="week date",how="outer")
sales_data_naitonal['week date']=sales_data_naitonal['week date'].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


In [39]:
data_media_national_wide

,week date,Cinema_impression,Circulation_impression,Digital_impression,Email_impression,Radio_impression,TV_impression,Cinema_click,Circulation_click,Digital_click,Email_click,Radio_click,TV_click,Cinema_cost,Circulation_cost,Digital_cost,Email_cost,Radio_cost,TV_cost
97,2018-08-05,0.0,0.0,14242556.0,89381577.0,0.0,0.0,0.0,0.0,498857.0,1023718.0,0.0,0.0,0.0,0.000000,1.298103e+05,40738.739038,0.0,0.000000
98,2018-08-12,0.0,0.0,25069418.0,103576959.0,0.0,1132248.0,0.0,0.0,657504.0,1028718.0,0.0,0.0,0.0,0.000000,2.393811e+05,47208.774389,0.0,44723.796000
99,2018-08-19,0.0,13481187.0,24786808.0,115447474.0,0.0,6603244.0,0.0,0.0,710012.0,1462263.0,0.0,0.0,0.0,739869.809954,2.229904e+05,52619.171353,0.0,139026.948500
100,2018-08-26,0.0,0.0,43110680.0,94261451.0,0.0,36609196.0,0.0,0.0,996617.0,1036578.0,0.0,0.0,0.0,0.000000,4.406212e+05,42962.910060,0.0,640585.074000
101,2018-09-02,0.0,15875846.0,43781261.0,92292476.0,0.0,34261057.0,0.0,0.0,850350.0,875504.0,0.0,0.0,0.0,819405.882640,4.140577e+05,42065.481738,0.0,576567.574500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,2020-06-28,0.0,0.0,68468259.0,166087949.0,0.0,4404505.5,0.0,0.0,1270996.0,1899069.0,0.0,0.0,0.0,0.000000,3.876314e+05,33982.816681,0.0,185556.918643
197,2020-07-05,0.0,0.0,81105258.0,162159442.0,0.0,0.0,0.0,0.0,1233252.0,1910490.0,0.0,0.0,0.0,0.000000,4.657328e+05,33179.015238,0.0,0.000000
198,2020-07-12,0.0,11960954.0,105303030.0,150712949.0,0.0,0.0,0.0,0.0,1270347.0,1741837.0,0.0,0.0,0.0,518102.625425,6.960930e+05,30836.978530,0.0,0.000000
199,2020-07-19,0.0,0.0,157619512.0,157237129.0,0.0,0.0,0.0,0.0,1315459.0,1512856.0,0.0,0.0,0.0,0.000000,1.002740e+06,32171.873772,0.0,0.000000


In [40]:
data_media_national_wide=pd.merge(data_media_national_wide,sales_data_naitonal,on="week date",how="left")
data_submedia_national_wide=pd.merge(data_submedia_national_wide,sales_data_naitonal,on="week date",how="left")

# Promotion

In [41]:
Rewards_Promotion_list=[datetime.date(2016,10,1),datetime.date(2016,10,2),
                        datetime.date(2017,1,21),datetime.date(2017,1,22),
                        datetime.date(2017,4,1),datetime.date(2017,4,2),
                        datetime.date(2017,7,8),datetime.date(2017,7,9),
                        datetime.date(2017,9,30),datetime.date(2017,10,1),
                        datetime.date(2018,1,20),datetime.date(2018,1,21),
                        datetime.date(2018,4,7),datetime.date(2018,4,8),
                        datetime.date(2018,7,7),datetime.date(2018,7,8),
                        datetime.date(2018,9,29),datetime.date(2018,9,30),
                        datetime.date(2019,1,26),datetime.date(2019,1,27),  
                        datetime.date(2019,4,6),datetime.date(2019,4,7),
                        datetime.date(2019,7,13),datetime.date(2019,7,14),
                        datetime.date(2020,1,18),datetime.date(2020,1,19)
                        # Q1 2020 F&F canceled due to COVID, according to the 2020 Marcom Calendar_11.11.xlsx
                        # Don't see the Q2 F&F event neither in the same excel above
                        
                       ]

df_Rewards_Promotion=pd.DataFrame({"Date":Rewards_Promotion_list},index=range(len(Rewards_Promotion_list)))
df_Rewards_Promotion['weekday']=df_Rewards_Promotion['Date'].apply(lambda x: x.weekday())
df_Rewards_Promotion['week date']=np.where(df_Rewards_Promotion['weekday']==6,df_Rewards_Promotion['Date'],df_Rewards_Promotion['Date']-datetime.timedelta(days=6))

del df_Rewards_Promotion['Date']

df_Rewards_Promotion_Sunday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==6]
df_Rewards_Promotion_Sunday['Sunday_rewards_ind']=1
del df_Rewards_Promotion_Sunday['weekday']
df_Rewards_Promotion_Sunday['week date']=df_Rewards_Promotion_Sunday['week date'].astype(str)


df_Rewards_Promotion_Saturday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==5]
df_Rewards_Promotion_Saturday['Saturday_rewards_ind']=1
del df_Rewards_Promotion_Saturday['weekday']
df_Rewards_Promotion_Saturday['week date']=df_Rewards_Promotion_Saturday['week date'].astype(str)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [42]:
holiday_1_week_df=pd.DataFrame({"week date":[datetime.date(2016,12,18),datetime.date(2017,12,17),datetime.date(2018,12,23),datetime.date(2019,12,22)],"Holiday_1_week_only_Ind":[1]*4},index=[0,1,2,3])
holiday_5_weeks_df=pd.DataFrame({"week date":[datetime.date(2016,12,18)-datetime.timedelta(days=x*7) for x in range(5)] +\
                                 [datetime.date(2017,12,17)-datetime.timedelta(days=x*7) for x in range(5)]+\
                                 [datetime.date(2018,12,23)-datetime.timedelta(days=x*7) for x in range(5)]+\
                                 [datetime.date(2019,12,22)-datetime.timedelta(days=x*7) for x in range(5)],
                                 "Holiday_5_weeks_only_Ind":[1]*20},index=[x for x in range(20)])

holiday_1_week_df['week date']=holiday_1_week_df['week date'].astype(str)
holiday_5_weeks_df['week date']=holiday_5_weeks_df['week date'].astype(str)

df_special_weeks_ind=pd.merge(df_Rewards_Promotion_Saturday,df_Rewards_Promotion_Sunday,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_1_week_df,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_5_weeks_df,on="week date",how="outer")


In [43]:
data_media_national_wide=pd.merge(data_media_national_wide,df_special_weeks_ind,on="week date",how="left")

data_submedia_national_wide=pd.merge(data_submedia_national_wide,df_special_weeks_ind,on="week date",how="left")

data_media_national_wide=data_media_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.fillna(0)

In [44]:
# Quarter label
week_start_1st_2018Q1=datetime.date(2018,2,4)

def quarter_of_FY(input_str_x):
    date_x=datetime.datetime.strptime(input_str_x,"%Y-%m-%d").date()
    quarter_diff=int(np.floor(((date_x-week_start_1st_2018Q1).days)/(7*13)))
    year_diff=int(np.floor(((date_x-week_start_1st_2018Q1).days)/(7*52)))
    year=year_diff+week_start_1st_2018Q1.year
    quarter=1+quarter_diff%4
    return "Q%i_%i"%(quarter,year)

In [45]:
for i in range(200):
    input_x=week_start_1st_2018Q1+datetime.timedelta(days=i*7)
    input_x=str(input_x)
    print(str(input_x),quarter_of_FY(input_x))

2018-02-04 Q1_2018
2018-02-11 Q1_2018
2018-02-18 Q1_2018
2018-02-25 Q1_2018
2018-03-04 Q1_2018
2018-03-11 Q1_2018
2018-03-18 Q1_2018
2018-03-25 Q1_2018
2018-04-01 Q1_2018
2018-04-08 Q1_2018
2018-04-15 Q1_2018
2018-04-22 Q1_2018
2018-04-29 Q1_2018
2018-05-06 Q2_2018
2018-05-13 Q2_2018
2018-05-20 Q2_2018
2018-05-27 Q2_2018
2018-06-03 Q2_2018
2018-06-10 Q2_2018
2018-06-17 Q2_2018
2018-06-24 Q2_2018
2018-07-01 Q2_2018
2018-07-08 Q2_2018
2018-07-15 Q2_2018
2018-07-22 Q2_2018
2018-07-29 Q2_2018
2018-08-05 Q3_2018
2018-08-12 Q3_2018
2018-08-19 Q3_2018
2018-08-26 Q3_2018
2018-09-02 Q3_2018
2018-09-09 Q3_2018
2018-09-16 Q3_2018
2018-09-23 Q3_2018
2018-09-30 Q3_2018
2018-10-07 Q3_2018
2018-10-14 Q3_2018
2018-10-21 Q3_2018
2018-10-28 Q3_2018
2018-11-04 Q4_2018
2018-11-11 Q4_2018
2018-11-18 Q4_2018
2018-11-25 Q4_2018
2018-12-02 Q4_2018
2018-12-09 Q4_2018
2018-12-16 Q4_2018
2018-12-23 Q4_2018
2018-12-30 Q4_2018
2019-01-06 Q4_2018
2019-01-13 Q4_2018
2019-01-20 Q4_2018
2019-01-27 Q4_2018
2019-02-03 Q

In [46]:
data_media_national_wide['BL_quarter']=data_media_national_wide['week date'].apply(quarter_of_FY)
data_submedia_national_wide['BL_quarter']=data_submedia_national_wide['week date'].apply(quarter_of_FY)

In [47]:
os.getcwd()

'/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q2'

In [48]:
data_media_national_wide.to_csv("./output/BL_MMM_data_media_national_wide_2020Q2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
data_submedia_national_wide.to_csv("./output/BL_MMM_data_submedia_national_wide_2020Q2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
print(datetime.datetime.now())

# Gonna QC for the last 3 quarters

2020-12-16 21:53:27.065022
